# LOCA2 Moving Averages

In [ ]:
##########################################################
#
# Library Calls.
#

# loading numpy

import numpy             as np

# loading matplotlib

import matplotlib.pyplot as plt

# loading xarray

import xarray            as xr

# Loading pandas

import pandas            as pd



import subprocess as subprocess



def geo_idx(dd, dd_array):
   """
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
    """
   geo_idx = (np.abs(dd_array - dd)).argmin()
   return geo_idx
#
##########################################################

##  Inventory and Model/Member Lookup Table

In [ ]:
##########################################################
#
# Inventory and Model/Member Lookup Table
#

root_directory       = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Climate_CONUS/Monthly/"
root_url             = "http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Climate_CONUS/Monthly/"

loca2_complete_file  = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/LOCA2_Model_Member_Complete_List.csv"

loca2_complete_list = pd.read_csv(filepath_or_buffer = loca2_complete_file)


modelsc               = loca2_complete_list[         "Model"].values
membersc              = loca2_complete_list[        "Member"].values
model_membersc        = loca2_complete_list[  "Model_Member"].values







model_member_key0 = np.array(modelsc+"."+membersc, dtype="str")
model_member_key =  np.array(["NULL"], dtype="str")

model_members_to_save = np.append(0,model_membersc)

model_member_key = np.append(model_member_key,model_member_key0)


model_member_key    = xr.DataArray(model_member_key, 
                                   coords={"model_member":model_members_to_save},
                                   name  = "model_member_key",
                                   dims  = ["model_member"],
                                   attrs = {"description" : "Model and Member Label",
                                              "long_name" : "Model and Member Label",
                                               "comment1" :   "LUT Indexing Starts at 0"})



display(model_member_key)

#
##########################################################

## File and Inventory Control

In [ ]:
##########################################################
#
# File and Inventory Control
#

root_directory       = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Climate_CONUS/Monthly/"
root_url             = "http://kyrill.ias.sdsmt.edu:8080/thredds/dodsC/LOCA2/Climate_CONUS/Monthly/"

loca2_inventory_file = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Original_CONUS/LOCA2_Model_Member_Available_List.csv"

loca2_ensembles_list = pd.read_csv(filepath_or_buffer = loca2_inventory_file)

loca2_ensembles_list = loca2_ensembles_list.query('Rank == 1')

models               = loca2_ensembles_list[         "Model"].values
members              = loca2_ensembles_list[        "Member"].values
model_members        = loca2_ensembles_list[  "Model_Member"].values
ranks                = loca2_ensembles_list[          "Rank"].values
n_complete_enss      = loca2_ensembles_list["n_complete_ens"].values
historical_invs      = loca2_ensembles_list[    "historical"].values
ssp245_invs          = loca2_ensembles_list[        "ssp245"].values
ssp370_invs          = loca2_ensembles_list[        "ssp370"].values
ssp585_invs          = loca2_ensembles_list[        "ssp585"].values

prec_invs = loca2_ensembles_list[    "pr"].values
tmax_invs = loca2_ensembles_list["tasmax"].values
tmin_invs = loca2_ensembles_list["tasmin"].values

scenarios            = ["historical", 
                            "ssp245", 
                            "ssp370", 
                            "ssp585"]







display(loca2_ensembles_list)

#
##########################################################


rank = str(ranks[3]).zfill(2)
display(rank)

# Pull Geospatial Metadata

In [ ]:

print(rank)
#
##########################################################

## Loop for Resultsmodel_member_key.values

In [ ]:
##########################################################
#
# Loop Test
#

root_directory_A       = "/data/DATASETS/LOCA_MACA_Ensembles/LOCA2/LOCA2_CONUS/Climate_CONUS/Annual"
    
for scenario in scenarios[1:]:
    print("# ################################################")
    First = True
    for m in range(len(models)-1):
        print("# ------------------------------------------------")

        model          =          models[m]
        member         =         members[m]
        model_member   =   model_members[m]
        rank           =   str(ranks[m]).zfill(2)

        n_complete_ens = n_complete_enss[m]
        prec_inv       =       prec_invs[m]
        tmax_inv       =       tmax_invs[m]
        tmin_inv       =       tmin_invs[m]

        model_member_name = np.array([model + "." + member], dtype="str")
        model_member      = np.array([model_member], dtype="int16").flatten()

        model_member = xr.DataArray(data   = model_member,
                                    coords = {"model_member":model_member},
                                    name   =  "model_member",
                                    dims   = ["model_member"],
                                    attrs  = {"description" : "Model and Member Code",
                                              "long_name"   : "Model and Member Code",
                                              "code_to_name_lookup_table":  model_member_key.values.tolist(),
                                              "comment1"    : "LUT Indexing Starts at 0"})

        file_url_h = root_directory + "/" + scenarios[0] + "/LOCA2-CONUS-ANNUAL_MIN___tasmin___" + model + "." + member + "___" + scenarios[0] + ".nc"

        print("# ================================================")

        inventory = loca2_ensembles_list.iloc[m].loc[scenario]
        print("# " + str(model_member[0].values).zfill(3) + " " + model + " " + member + " " + scenario + " " + inventory)

        if (inventory != "---"):


            if ("N" in inventory):

                variable    = "tasmin"

                print("#  . . . . . . . . . . . . . . . . . . . . . . . .")

                hist_file         = root_directory_A                          +  "/"  + \
                                    "historical"                              +  "/"  + \
                                    "LOCA2-CONUS-ANNUAL_MIN"                  + "___" + \
                                    variable                                  + "___" + \
                                    loca2_ensembles_list.at[m,"Model"]        +  "."  + \
                                    loca2_ensembles_list.at[m,"Member"]       + "___" + \
                                    "historical"                              + ".nc"  

                futr_file         = root_directory_A                          +  "/"  + \
                                    scenario                                  +  "/"  + \
                                    "LOCA2-CONUS-ANNUAL_MIN"                  + "___" + \
                                    variable                                  + "___" + \
                                    loca2_ensembles_list.at[m,"Model"]        +  "."  + \
                                    loca2_ensembles_list.at[m,"Member"]       + "___" + \
                                    scenario                                  + ".nc"  

                combined_file     = root_directory_A                          +  "/"  + \
                                    "LOCA2-CONUS-ANNUAL30YRUNMEAN_MIN"        + "___" + \
                                    variable                                  + "___" + \
                                    str(model_member[0].values).zfill(3)      + "___" + \
                                    loca2_ensembles_list.at[m,"Model"]        +  "."  + \
                                    loca2_ensembles_list.at[m,"Member"]       + "___" + \
                                    scenario                                  + ".nc"  
 
                combined_wc_files = root_directory_A                          +  "/"  + \
                                    "LOCA2-CONUS-ANNUAL30YRUNMEAN_MIN"        + "___" + \
                                    variable                                  + "___" + \
                                    "???"                                     + "___" + \
                                    "*"                                       + "___" + \
                                    scenario                                  + ".nc" 

                final_merged_file = root_directory_A                          +  "/"  + \
                                    "LOCA2-CONUS-ANNUAL30YRUNMEAN_MIN"        + "___" + \
                                    variable                                  + "___" + \
                                    "ALLRANK"+ rank                           + "___" + \
                                    scenario                                  + ".nc" 
        
                if (First):
                    model_member_array = np.array(model_members[m], dtype = "int16")
                    First              = False
                else:
                    model_member_array = np.append(model_member_array, model_members[m]).flatten()
                    
                    

                cdo_cat_command = "nohup cdo --no_history -f nc4 -z zip_9 cat "

                command_aggregate = cdo_cat_command + hist_file + " " + futr_file + " ./deleteme.nc"
                subprocess.run(["rm -fr ./deleteme.nc"], shell = True, check = True)
                subprocess.run([command_aggregate], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a bounds,time,d,, ./deleteme.nc"], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a bounds,lon,d,, ./deleteme.nc"], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a bounds,lat,d,, ./deleteme.nc"], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a _FillValue,lon,d,, ./deleteme.nc"], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a _FillValue,lat,d,, ./deleteme.nc"], shell = True, check = True)
                subprocess.run(["ncatted -Oh -a _FillValue,time,d,, ./deleteme.nc"], shell = True, check = True)
                xf_h = xr.open_dataset(filename_or_obj = "./deleteme.nc")
                tasmin0 = xf_h["tasmin"]
                subprocess.run(["rm -fr ./deleteme.nc"], shell = True, check = True)



                tasmin = tasmin0.rolling(time   =   30, 
                                         center = True).mean().dropna(dim = "time", 
                                                                      how =  "all")
                
                tasmin.name = "tasmin"
                tasmin.attrs["cell_methods"] = "time: minimum within days  time: minimum within years  time: mean over 30 years " 
                
                
                
                tasmin = tasmin.expand_dims(dim={"model_member" : 1})              

         
                outdata = xr.Dataset(data_vars = {"tasmin"     :       tasmin,
                                                  "model_member": model_member},
                                     attrs     = {"scenario"   :     scenario})

                outdata.to_netcdf(path           = combined_file, 
                                  mode           =           'w', 
                                  format         =     "NETCDF4",
                                  engine         =    "h5netcdf", #
                                  unlimited_dims = "model_member",
                                  encoding       = {"tasmin": {       "dtype": "int16", 
                                                               "scale_factor":    0.1,
                                                                "add_offset" :    0.0,                                        
                                                                 "_FillValue": -32767}})
                
                subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && ncatted -Oh -a _FillValue,lon,d,, " + combined_file], 
                               shell = True, 
                               check = True)
                subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && ncatted -Oh -a _FillValue,lat,d,, " + combined_file], 
                               shell = True, 
                               check = True)
                subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && ncatted -Oh -a _FillValue,time,d,, " + combined_file],  
                               shell = True, 
                               check = True)
            # end check on avaiable variable
        # end check on on avaiable member
            
    #end loop on model

    print("# = = = = = = = = = = = = = = = = = = = = = = = = ") 
    
    
    model_member = xr.DataArray(data   = model_member_array,
                                name   =  "model_member",
                                dims   = ["model_member"],
                                attrs  = {"description" : "Model and Member Code",
                                          "long_name"   : "Model and Member Code",
                                          "code_to_name_lookup_table":  model_member_key.values.tolist(),
                                          "comment1"    : "LUT Indexing Starts at 0"})

    model_member_ds = xr.Dataset(data_vars = {"model_member"          : model_member},
                                 attrs     = {"scenario"   :     scenario})
    
    model_member_ds.to_netcdf(path           = "./model_member.nc", 
                               mode           =           'w', 
                               format         =     "NETCDF4",
                               engine         =    "h5netcdf", #
                               unlimited_dims = "model_member")  
        
    cdo_cat_command = "nohup cdo --no_history -f nc4 -z zip_9 cat "
    nco_cat_command = "nohup ncecat -M -u model_member "
    command_aggregate = cdo_cat_command +combined_wc_files + " ./temp.nc"     
    print("# Final Aggregation")
    subprocess.run(["rm -fr " + final_merged_file + " temp.nc temp2.nc"], 
                   shell = True, 
                   check = True)    
    subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && " + command_aggregate], 
                   shell = True, 
                   check = True)
    print("# Files Concatenated")

    subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && ncks -C -O -x -v model_member ./temp.nc " + final_merged_file], 
                   shell = True, 
                   check = True)
    print("# dimension dropped")
    subprocess.run(["export HDF5_USE_FILE_LOCKING=FALSE && ncks -h -A ./model_member.nc " + final_merged_file], 
                   shell = True, 
                   check = True)
    print("# dimension swapped")
    subprocess.run(["rm -fr  ./temp.nc ./temp2.nc ./ ./model_member.nc" + combined_wc_files], 
                   shell = True, 
                   check = True) 

# end loop on scenario
                
print("# ================================================")
print("end processing")

# Version History

In [ ]:
################################################################
#
# Loading Version Information
#

%load_ext version_information
%version_information version_information numpy, matplotlib, xarray, pandas, cartopy, metpy

#
################################################################

In [ ]:
print("# " + str(model_member[0]).zfill(3) + " " + model + " " + member + " " + scenario + " " + inventory)
